# XGBoost

In [1]:
import os
import datetime
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
if 'current_dir' in locals():
    print("Directory has been set before:")
else:
    current_dir = os.getcwd()[:-9] 
print(current_dir)
os.chdir(current_dir + '/scripts')
import preprocessing
os.chdir(current_dir)
submission = pd.read_csv(current_dir + 'submissions/submission_v1.csv') #submission as template (version is not important)

C:\Users\suhai\Downloads\Customer-Loyalty\


In [9]:
train_x, train_l, testset_x = preprocessing.load_data(features="custom", custom_features=["train_feature_fam.csv","test_feature_fam.csv"])

C:\Users\suhai\Downloads\Customer-Loyalty
this is a train feature : train_feature_fam.csv
this is a test feature : test_feature_fam.csv


## Training
Training will be done in the following cells. First a random part of the training data is assigned as training set and the leftovers will be the validation set. By making the _trainsize_ variable round(train.shape[0]/1), the entirety of the training data is used as training set, so no evaluation will be done in the section _Testing within training set_.

In [10]:
# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
# fit model no training data
# model = XGBClassifier()
# model.fit(train_x, train_l)

dtrain = xgb.DMatrix(train_x, train_l)
watchlist = [(dtrain, 'train')]
xgb_pars = {'min_child_weight': 0, 'eta': 1.0,'colsample_bytree': 0.9, 
        'max_depth': 100, 'subsample': 1., 'lambda': 0., 'nthread': 10, 'booster' : 'gbtree', 'silent': 1, 'eval_metric': 'rmse', 'objective': 'reg:linear'}
model = xgb.train(xgb_pars, dtrain, 10, watchlist, early_stopping_rounds=10, maximize=False, verbose_eval=1)
print('Modeling RMSLE %.5f' % model.best_score)

## =====DEFAULT/COPIED PARAMETERS=====
# dtrain = xgb.DMatrix(train_x, train_l)
# watchlist = [(dtrain, 'train')]
# xgb_pars = {'min_child_weight': 1, 'eta': 0.5, 'colsample_bytree': 0.9, 
#         'max_depth': 6, 'subsample': 0.9, 'lambda': 1., 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'eval_metric': 'rmse', 'objective': 'reg:linear'}
# model = xgb.train(xgb_pars, dtrain, 100, watchlist, early_stopping_rounds=10, maximize=False, verbose_eval=1)
# print('Modeling RMSLE %.5f' % model.best_score)

[0]	train-rmse:3.83853
Will train until train-rmse hasn't improved in 10 rounds.
[1]	train-rmse:3.8378
[2]	train-rmse:3.83761
[3]	train-rmse:3.8346
[4]	train-rmse:3.8346
[5]	train-rmse:3.8346
[6]	train-rmse:3.8346
[7]	train-rmse:3.8346
[8]	train-rmse:3.8346
[9]	train-rmse:3.8346
Modeling RMSLE 3.83460


## Predictions for test set (creating a submission .csv file)
Make sure to use the entire training set in the the "Training" section

In [12]:
print(testset_x)
testset_x = xgb.DMatrix(testset_x)
# make predictions for test data
y_pred = model.predict(testset_x)
predictions = [round(value) for value in y_pred]
submission.target = predictions
# # evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

[[3 3 1 65]
 [2 3 0 62]
 [5 1 1 69]
 ...
 [5 1 1 58]
 [2 1 0 67]
 [5 1 1 59]]


In [13]:
# submission.target = submission_predictions

for i in range(1,100,1):
    if os.path.isfile('submissions/submission_v0-00' + str(i) + '.csv') == False:
        submission.to_csv('submissions/submission_v0-00' + str(i) + '.csv', index = False)
        print('Submission was saved under the name:' + '\033[1m' ' submission_v0-00' + str(i) + '.csv' + '\033[0;0m')
        break

# [Errno 13] Permission denied: 'submissions/submission_v0-001.csv'
#        means that the file is still opened somewhere

Submission was saved under the name: submission_v0-005.csv


In [14]:
# run this after previous cel to see the submission
submission.target.value_counts()

-0.0     79115
-1.0     43992
-2.0       375
 1.0        73
-3.0        33
-4.0        16
-6.0         9
-5.0         5
-17.0        3
 2.0         2
Name: target, dtype: int64